# Coronavirus Latest
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide_colab_badge: true
- comments: false
- permalink:/covid-overview/


In [0]:
#hide
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML

base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'

# confirmed, deaths, recovered
def get_time_series(type):
  _url = f'{base_url}/csse_covid_19_time_series/time_series_covid19_{type}_global.csv'
  _dff = pd.read_csv(_url)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

df_confirmed = get_time_series('confirmed')
df_recovered = get_time_series('recovered')
df_deaths = get_time_series('deaths')
date_latest = df_confirmed['Date'].max()

df_last = lambda name, _df, n: _df[_df['Date'].isin(_df['Date'].tail(n))].set_index(['Country', 'Date']).rename(columns={'Cases': name})
df_summary = lambda last_n: pd.concat([df_last('Confirmed', df_confirmed, last_n), df_last('Deaths', df_deaths, last_n), df_last('Recovered', df_recovered, last_n)], axis=1)
df_global = df_summary(2).groupby(['Date']).sum().reset_index()
s_date_latest = date_latest.strftime("%B %d, %Y")
n_confirmed = df_global.loc[1]['Confirmed']
n_deaths = df_global.loc[1]['Deaths']
n_recovered = df_global.loc[1]['Recovered']

df_global_diff = df_global.diff()
n_confirmed_diff = df_global_diff.loc[1]['Confirmed']
n_deaths_diff = df_global_diff.loc[1]['Deaths']
n_recovered_diff = df_global_diff.loc[1]['Recovered']
countries = df_confirmed['Country'].unique()

In [19]:
#hide_input
HTML(
    f'<div style="font-size: 1.0em;">'
    f'<div> \
        <span style="font-size: 2.0em;">{n_confirmed:,}</span><br/>\
        Total cases reported worldwide, {s_date_latest} \
      </div>\
      <br/>'
    f'<div><span style="font-size: 2.0em;">{n_confirmed_diff:,.0f}</span><br/>New cases reported since last day</div><br/>'
    f'<div><span style="font-size: 2.0em;">{n_recovered:,}</span> +{n_recovered_diff:,.0f} ({n_recovered/n_confirmed*100:.2f}%)<br/>Total recovered reported worldwide</div><br/>'
    f'<div><span style="font-size: 2.0em;">{n_deaths:,}</span> +{n_deaths_diff:,.0f} ({n_deaths/n_confirmed*100:.2f}%)<br/>Total deaths reported worldwide</div><br/>'
    f'<div><span style="font-size: 2.0em;">{len(countries) - 1}</span><br/>Total number of countries tracked</div><br/>'
    f'</div>'
)

In [11]:
#hide_input
df_confirmed_latest = df_confirmed.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).tail(2)
df_confirmed_latest = df_confirmed_latest.reset_index(drop=True).transpose()
df_confirmed_latest['New'] = df_confirmed_latest.diff(axis=1)[1]
df_confirmed_latest['Change'] = df_confirmed_latest.pct_change(axis=1)[1]

df_confirmed_pct_mean = df_confirmed.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).pct_change().rolling(window=7).mean()
df_confirmed_pct_mean_latest = df_confirmed_pct_mean[-1:].reset_index(drop=True).transpose().sort_values(by=0,ascending=False).rename(columns={0: 'Change_Mean_7D'})
df_confirmed_latest = pd.merge(df_confirmed_latest, df_confirmed_pct_mean_latest, on = 'Country')
df_confirmed_latest['Trending'] = df_confirmed_latest['Change'] - df_confirmed_latest['Change_Mean_7D']

df_confirmed_latest = df_confirmed_latest.drop(columns=0).rename(columns={1: 'Total'}).sort_values(by=['Change', 'New', 'Trending'], ascending=False)
df_confirmed_major = df_confirmed_latest[(df_confirmed_latest['New'] > 50) & (df_confirmed_latest['Total'] > 1000)].reset_index()

alt.Chart(df_confirmed_major).mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('Change_Mean_7D:Q', axis=alt.Axis(format='%'), title='Mean Change'),
    tooltip=list(df_confirmed_major)
).properties(
    width=600,
    title='Percentage Change (Mean of 7 Days) of Confirmed Case'
)

alt.Chart(...)

In [12]:
#hide
df_confirmed_latest.sort_values(by=['Change'], ascending=False)

,Total,New,Change,Change_Mean_7D,Trending
Country,,,,,
Cabo Verde,56,45.0,4.090909,0.654824,3.436085
Congo (Brazzaville),117,57.0,0.950000,0.183333,0.766667
Jamaica,125,52.0,0.712329,0.123369,0.588960
Tanzania,88,35.0,0.660377,0.221894,0.438483
Gabon,80,23.0,0.403509,0.138795,0.264714
...,...,...,...,...,...
Monaco,93,0.0,0.000000,0.020222,-0.020222
Cameroon,848,0.0,0.000000,0.022491,-0.022491
Mauritania,7,0.0,0.000000,0.023810,-0.023810


In [13]:
#hide
df_confirmed_countries = df_confirmed.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).transpose()
df_confirmed_countries.loc['France']

Date
2020-01-22         0
2020-01-23         0
2020-01-24         2
2020-01-25         3
2020-01-26         3
               ...  
2020-04-11    130727
2020-04-12    133670
2020-04-13    137875
2020-04-14    131361
2020-04-15    134582
Name: France, Length: 85, dtype: int64